In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls

data_description.txt  house.ipynb  sample_submission.csv  test.csv  train.csv


In [0]:
import numpy as np 
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

In [0]:
train=pd.read_csv("train.csv")
train.head()
test=pd.read_csv("test.csv")

In [8]:
print('th train data has {} rows and {} features'.format(train.shape[0],train.shape[1]))
print('the test data has {} rows and {} features'.format(test.shape[0],test.shape[1]))

th train data has 1460 rows and 81 features
the test data has 1459 rows and 80 features


In [9]:
data=pd.concat([train.iloc[:,:-1],test],axis=0)
print('tha data has {} rows and {} features'.format(data.shape[0],data.shape[1]))

tha data has 2919 rows and 80 features


In [10]:
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 80 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null int64
MSZoning         2915 non-null object
LotFrontage      2433 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            198 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2917 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2918 non-

In [0]:
data = data.drop(columns=['Id'],axis=1)
num_features=data.select_dtypes(include=['int64','float64'])
categorical_features=data.select_dtypes(include='object')
num_features.describe()

categorical_features.describe()

data.isnull().sum().sort_values(ascending=False)[:34]
#print(categorical_features.isnull().sum().sort_values(ascending=False)[:23])
#num_features.isnull().sum().sort_values(ascending=False)[:11]

f = open("data_description.txt", "r")
#print(f.read())



In [0]:
#data['LotFrontage'].fillna(int(data['LotFrontage'].mean()),inplace=True)
data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
data['LotFrontage'].isnull().sum()

#create a new class 'other'
features=['Functional','Utilities','Electrical','KitchenQual','SaleType','Exterior2nd','Exterior1st','PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCond','GarageQual','GarageFinish','GarageType','BsmtCond','BsmtExposure','BsmtQual','BsmtFinType2','BsmtFinType1','MasVnrType']
for name in features:
    data[name].fillna('Other',inplace=True)
    
    

In [0]:
data[features].isnull().sum()
data['MSZoning'] = data.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))


In [15]:
#use the mode value for the rest of the features
"""mode=['MSZoning','Functional','Utilities','Electrical','KitchenQual','SaleType','Exterior2nd','Exterior1st']
for name in mode:
    data[name].fillna(data[name].mode()[0],inplace=True)"""


"mode=['MSZoning','Functional','Utilities','Electrical','KitchenQual','SaleType','Exterior2nd','Exterior1st']\nfor name in mode:\n    data[name].fillna(data[name].mode()[0],inplace=True)"

In [0]:
zero=['GarageYrBlt','GarageArea','MasVnrArea','BsmtHalfBath','BsmtHalfBath','BsmtFullBath','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','GarageCars']
for name in zero:
    data[name].fillna(0,inplace=True)

In [17]:
data.isnull().sum().sum()
data.select_dtypes(include='object').columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [0]:
def dummies(d):
    dummies_df=pd.DataFrame()
    for name in d.select_dtypes(include='object').columns:
        dummies = pd.get_dummies(d[name], drop_first=False)
        dummies = dummies.add_prefix("{}_".format(name))
        dummies_df=pd.concat([dummies_df,dummies],axis=1)
    return dummies_df

In [0]:
dummies_data=dummies(data)

In [20]:
dummies_data.shape

(2919, 273)

In [0]:
data['MSSubClass'] = data['MSSubClass'].astype(str)

In [0]:
MSSubClass_dummmies=pd.get_dummies(data['MSSubClass'], drop_first=False).add_prefix("MSSubClass_")

In [0]:
object_features=['MSSubClass','MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition']

In [0]:
data_drop=data.drop(columns=object_features,axis=1)
data_drop.shape
data_drop.columns
from scipy import stats
from scipy.stats import pointbiserialr, spearmanr, skew, pearsonr

for col in data_drop.columns:
    if skew(data_drop[col]) > 0.75:
        data_drop[col] = np.log1p(data_drop[col])
    

In [25]:
final_data=pd.concat([MSSubClass_dummmies,data_drop,dummies_data],axis=1)
final_data.shape
#Re-spliting the data into train and test data set and ignore the Id column
train_df=final_data.iloc[:1460,1:]
test_data=final_data.iloc[1460:,1:]
print(train_df.shape)
test_data.shape

(1460, 323)


(1459, 323)

In [0]:
#Concatenating the train_df and the target variable from the original train data set 
train_data=pd.concat([train_df,train.iloc[:,-1]],axis=1)
#train_data.shape

In [27]:
"""from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.preprocessing import MinMaxScaler"""

'from sklearn.model_selection import cross_val_score, GridSearchCV\nfrom sklearn.ensemble import RandomForestRegressor\n#from sklearn.preprocessing import MinMaxScaler'

In [0]:
#spliting the train_data into X: independent variables and y: target variable
X=train_data.iloc[:,:-1]
y=train_data.iloc[:,-1]

In [0]:
from sklearn.linear_model import Ridge, RidgeCV, LassoCV

In [30]:
"""las_cv = LassoCV(alphas=(0.0001, 0.0005, 0.001, 0.01, 0.05, 0.1, 0.3, 1, 3, 5, 10))
las_cv.fit(X,y)
las_cv_preds=las_cv.predict(test_data)"""

'las_cv = LassoCV(alphas=(0.0001, 0.0005, 0.001, 0.01, 0.05, 0.1, 0.3, 1, 3, 5, 10))\nlas_cv.fit(X,y)\nlas_cv_preds=las_cv.predict(test_data)'

In [0]:
ridge_cv = RidgeCV(alphas=(0.01, 0.05, 0.1, 0.3, 1, 3, 5, 10))
ridge_cv.fit(X, y)
ridge_cv_preds=ridge_cv.predict(test_data)

In [0]:
"""ridge = Ridge(alpha=10, solver='auto')
ridge.fit(X, y)
predictions=ridge.predict(test_data)"""

In [32]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=340, max_depth=2, learning_rate=0.2)
model_xgb.fit(X, y)
xgb_preds=model_xgb.predict(test_data)
predictions = 0.5 * ridge_cv_preds + 0.5 * xgb_preds
#display the first 5 predictions of sale price
predictions[:5]

[17:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([117347.30588133, 162752.2398527 , 185864.73163177, 190148.26587855,
       202151.42710657])

In [33]:
#make the submission data frame
submission = {
    'Id': test.Id.values,
    'SalePrice': predictions
}
solution = pd.DataFrame(submission)
solution.head()

,Id,SalePrice
0,1461,117347.305881
1,1462,162752.239853
2,1463,185864.731632
3,1464,190148.265879
4,1465,202151.427107


In [0]:
#make the submission file
solution.to_csv('submission.csv',index=False)